In [ ]:
import matplotlib.pyplot as plt
from jax import numpy as jnp
import pandas as pd

from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter, DerivedOutput, Time, Function

In [ ]:
def build_demog_model(
    config: dict,
) -> CompartmentalModel:
    
    # Model characteristics
    compartments = (
        "Susceptible", 
        "Pre-infectious", 
        "Infectious", 
        "Immune"
    )
    model = CompartmentalModel(
        times=(
            config["start_time"] * 365.,
            config["end_time"] * 365.,
        ),
        compartments=compartments,
        infectious_compartments=["Infectious"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": config["total_population"] - config["infectious_seed"], 
            "Infectious": config["infectious_seed"],
        }
    )
    
    # Transitions
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("r0") / Parameter("infectious_period"),
        source="Susceptible", 
        dest="Pre-infectious"
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=1. / Parameter("latent_period"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / Parameter("infectious_period"), 
        source="Infectious", 
        dest="Immune",
    )
    model.add_universal_death_flows(
        "universal_death",
        death_rate=1. / Parameter("life_expectancy") / 365.,
    )
    model.add_crude_birth_flow(
        "births",
        1. / Parameter("life_expectancy") / 365.,
        "Susceptible",
    )
    
    # Outputs
    model.request_output_for_compartments(
        "total_population",
        compartments,
    )
    model.request_output_for_flow(
        name="incidence",
        flow_name="progression",
    )
    
    model.request_function_output(
        name="incidence_rate",
        func=DerivedOutput("incidence") / DerivedOutput("total_population") * 1e5,
    )

    # Vaccination stratification
    vacc_strat = Stratification(
        "vaccination",
        ["vaccinated", "unvaccinated"],
        ["Susceptible"],
    )
    vacc_strat.set_population_split(
        {
            "vaccinated": 0.,
            "unvaccinated": 1.,
        }
    )
    vacc_strat.set_flow_adjustments(
        flow_name="infection",
        adjustments={
            "vaccinated": 0.,
            "unvaccinated": 1.,
        },
    )
    
    def step_up(time, vacc_coverage):
        return jnp.where(time > 0., vacc_coverage, 0.)
    
    def step_down(time, vacc_coverage):
        return jnp.where(time > 0., 1. - vacc_coverage, 1.)
    
    vacc_strat.set_flow_adjustments(
        flow_name="births",
        adjustments={
            "vaccinated": Function(step_up, [Time, Parameter("vacc_coverage")]),
            "unvaccinated": Function(step_down, [Time, Parameter("vacc_coverage")]),
        },
    )
    model.stratify_with(vacc_strat)    

    return model

In [ ]:
model_config = {
    "total_population": 1e5,
    "infectious_seed": 1.,
    "start_time": -100.,
    "end_time": 60.,
}

parameters = {
    "r0": 13.,
    "latent_period": 8.,
    "infectious_period": 7.,
    "life_expectancy": 70.,
}

#### Panel b)

In [ ]:
coverage_values = (0.5, 0.8, 0.9)
vacc_model = build_demog_model(config=model_config)

outputs = pd.DataFrame(columns=coverage_values)
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))
line_styles = (":", "--", "-")
for i_cov, coverage in enumerate(coverage_values):
    parameters.update({"vacc_coverage": coverage})
    vacc_model.run(parameters=parameters)
    outputs[coverage] = vacc_model.get_derived_outputs_df()["incidence_rate"]
    left_ax.plot(
        outputs.index / 365.,
        outputs[coverage],
        color="k",
        linestyle=line_styles[i_cov],
        label=f"{int(coverage * 100)}%",
    )
    
left_ax.legend(title="% of newborns immunized:")
left_ax.set_xlabel("Time (years)")
left_ax.set_xlim(-10., model_config["end_time"])
left_ax.set_ylabel("New infectious persons/100,000/day")
left_ax.set_ylim(0., 5.)
left_ax.grid(axis="y")
left_ax.spines.top.set_visible(False)
left_ax.spines.right.set_visible(False)

image = plt.imread("./figures/fig_4_29a.jpg")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");